In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from datagen_pytorch import DataGenerator
from models import Resnet
from torchsummary import summary
from torch.utils.data import DataLoader
from LCFCNBinaryLoss import lcfcn_binary_loss
import torch
import torch.optim as optim
from torchvision.transforms import ToPILImage
import numpy as np
from og_loss import lc_loss
from utils_pytorch import visualize_dots

In [12]:
gen = DataGenerator(root='__misc/', augment=False, scale=4, split='train', debug_mode=True)

In [14]:
batch = gen.__getitem__(0)
print(np.asarray(batch['points']).shape)
print(gen.__len__())


(1000, 1500)
1


In [26]:
visualize_dots(np.asarray(batch['OG_image']), np.asarray(batch['points']))

In [5]:
dataloader = DataLoader(gen)

In [6]:
model = Resnet(n_classes=2, layers=101).cuda()

In [7]:
torch.manual_seed(1) 
torch.cuda.manual_seed_all(1)


In [8]:
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, weight_decay=0.0005)

In [ ]:
for epoch in range(100):
    loss_sum = 0.
    for batch in dataloader:
        opt.zero_grad()
        loss = lc_loss(model, batch)
        loss.backward()
        opt.step()
        
        loss_sum+=loss.item()
        
    print("\nepoch:{} loss:{}".format(epoch+1, loss_sum/len(dataloader)))
    out = model(batch['images'].cuda())
    out = torch.nn.functional.softmax(out,1)
    out_np = out.cpu().detach().numpy()[0]

    img = np.asarray(batch['OG_image'][0])
    visualize_dots(img, (out_np[1] > 0.5).astype(int), save=True, name=epoch)

In [16]:
out = model(batch['images'].cuda())

In [17]:
out = torch.nn.functional.softmax(out,1)

In [18]:
out_np = out.cpu().detach().numpy()[0]

In [19]:
img = np.asarray(batch['OG_image'][0])

In [20]:
visualize_dots(img, (out_np[1] > 0.5).astype(int), save=False)

In [15]:
gen = DataGenerator(root='__misc/', scale=5, split='test', augment=False)
batch = gen.__getitem__(0)


In [16]:
batch['images'].shape

torch.Size([3, 566, 851])

In [17]:
dataloader = DataLoader(gen)

In [18]:
for i,b in enumerate(dataloader):
    print(b['images'].shape)
    break

torch.Size([1, 3, 566, 851])


TypeError: 'int' object is not subscriptable

In [7]:
model = Resnet(n_classes=2, layers=101).cuda()

In [8]:
model.load_state_dict(torch.load('model_data/model/99.pth'))

In [19]:
model.eval()

for batch in dataloader:
    out = model(batch['images'].cuda())
    

In [20]:
out = torch.nn.functional.softmax(out,1)

In [21]:
out_np = out.cpu().detach().numpy()[0]

In [22]:
img = np.asarray(batch['OG_image'][0])

In [30]:
visualize_dots(img, (out_np[1] > 0.15).astype(int), save=False)